In [1]:
import os
%pwd


'd:\\Deep Learning Projects\\Deep-learning-project-mlflow-dvc\\Deep-Learning-Project-MLFlow-DVC\\research'

In [2]:
os.chdir("../") #change directory to one folder back
%pwd

'd:\\Deep Learning Projects\\Deep-learning-project-mlflow-dvc\\Deep-Learning-Project-MLFlow-DVC'

In [6]:
#Entity : return type of any function

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)   #dataclass is a deocrator that can be assignd to any function to make it an entity. Its  not exactly a class but behaves like a class
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion 

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

            

In [13]:
##prepare components

import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size


In [14]:
class DataIngestion:
    def __init__(self, config : DataIngestionConfig):
        self.config = config
    

    def download_file(self)-> str:
        '''
        Fetch data from google drive
        '''

        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading file from : {dataset_url} into : {zip_download_dir}")

            file_id = dataset_url.split('/')[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix + file_id, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
        
        except Exception as e:
            raise e

    def extract_zip_file(self):
        '''
        zip_file_path : str
        Extract zip file into directory
        function returns None
        '''

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            logger.info(f"Extracting zip file : {self.config.local_data_file} into dir : {unzip_path}")
            zip_ref.extractall(unzip_path)


In [15]:
## define pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-12-05 01:06:08,001: INFO: common: yaml_file: config\config.yaml loaded successfully]
[2025-12-05 01:06:08,007: INFO: common: yaml_file: params.yaml loaded successfully]
[2025-12-05 01:06:08,008: INFO: common: created directory at : artifacts]
[2025-12-05 01:06:08,009: INFO: common: created directory at : data_ingestion]
[2025-12-05 01:06:08,034: INFO: 2193935216: Downloading file from : https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into : artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=3ed1bac2-89ea-4c86-a1f6-ffcc331c6273
To: d:\Deep Learning Projects\Deep-learning-project-mlflow-dvc\Deep-Learning-Project-MLFlow-DVC\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [00:11<00:00, 5.13MB/s]

[2025-12-05 01:06:22,261: INFO: 2193935216: Downloaded data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]
[2025-12-05 01:06:22,327: INFO: 2193935216: Extracting zip file : artifacts/data_ingestion/data.zip into dir : artifacts/data_ingestion]
